In [7]:
import pandas as pd
import os

In [8]:
data_dir = '../data/argumentation/'
LOGIC_dir = '../data/LOGIC'

In [9]:
df_cckg = pd.read_csv(os.path.join(data_dir, 'train_cckg.tsv'), sep='\t')
df_iam = pd.read_csv(os.path.join(data_dir, 'train_iam.tsv'), sep='\t')

In [10]:
examples = pd.read_json(os.path.join(data_dir, 'fallacies_arguments_support.json'))
df_mapping = pd.read_csv(os.path.join(LOGIC_dir, 'mappings.csv'))
df_logic = pd.read_csv(os.path.join(LOGIC_dir, 'train.csv'))

In [11]:
def generate_prompt(topic, fallacy_type, supporting=True):
    example1 = examples[fallacy_type]['example1']
    example2 = examples[fallacy_type]['example2'] 
    fallacies = df_logic[df_logic['label'] == fallacy_type].sample(2)['text'].values
    text =  f"""You are given a topic.  
    Your task is to generate a {'supporting' if supporting else 'counter'} {fallacy_type} logical fallacy argument in the context of the topic. 
    It should not be longer than 25 words. 
    
    {fallacy_type} is defined as: {df_mapping[df_mapping['Original Name']==fallacy_type]['Description'].values[0]}
    An example of {fallacy_type} fallacy is: 
    {fallacies[0]}
    {fallacies[1]}
    
    Here is an example of a supporting {fallacy_type} fallacy: 
    {example1}
    
    return 
    {"{"}
        "topic": {topic},
        "fallacy type": {fallacy_type},
        "{fallacy_type} fallacy support": <> 
    {"}"}
    """
    return text

In [12]:
topic =  df_cckg.topic.sample(1).values[0]
fallacy_types = examples.keys()
fallacy_type = fallacy_types[12]
print(fallacy_type)
prompt = generate_prompt(topic=topic, fallacy_type=fallacy_type)
print(topic)
print()
print(prompt)

intentional
Assisted suicide is compassionate.

You are given a topic.  
    Your task is to generate a supporting intentional logical fallacy argument in the context of the topic. 
    It should not be longer than 25 words. 
    
    intentional is defined as: some intentional (sometimes subconscious) action/choice to incorrectly support an argument
    An example of intentional is: 
    ['Why must you always follow such an stupid and uninformed approach to life?']
    ['Why did you guys take so a stupid and uninformed approach to ethics?']
    
    Here is an example of a supporting intentional fallacies: 
    {'topic': 'Are aliens real', 'fallacy type': 'intentional fallacy', 'intentional fallacy support': "You can't prove that aliens don't exist, so they must be real"}
    return 
    {
        "topic": Assisted suicide is compassionate.,
        "fallacy type": intentional,
        "intentional fallacy support": <> 
    }
    
